# Introduction

In this tutorial, we will take a closer look at strategies in the context of both multi-step forecasting and simultaneous forecasting of multiple time series.

We will explore how strategies affect the feature generation process used for model training.

Let's import everything we need and define functions to obtain the results.

In [6]:
import warnings

warnings.filterwarnings("ignore")

from copy import deepcopy
from typing import List, Optional, Union

import numpy as np
import pandas as pd

from tsururu.dataset import IndexSlicer, Pipeline, TSDataset
from tsururu.model_training.trainer import DLTrainer, MLTrainer
from tsururu.model_training.validator import KFoldCrossValidator
from tsururu.models.boost import CatBoost
from tsururu.strategies import (
    DirectStrategy,
    FlatWideMIMOStrategy,
    MIMOStrategy,
    RecursiveStrategy,
)
from tsururu.strategies.base import Strategy
from tsururu.strategies.utils import timing_decorator
from tsururu.transformers import (
    DateSeasonsGenerator,
    DifferenceNormalizer,
    LagTransformer,
    LastKnownNormalizer,
    SequentialTransformer,
    StandardScalerTransformer,
    TargetGenerator,
    UnionTransformer,
)

index_slicer = IndexSlicer()

import time
from pathlib import Path
from typing import Callable, Dict, List, Optional, Tuple, Union

import numpy as np

from tsururu.models.base import Estimator
from tsururu.model_training.validator import Validator

try:
    import torch
    import torch.nn as nn
    from torch.utils.data import Subset
except ImportError:
    torch = None
    Subset = None
    nn = None

Let's transfer the necessary code into the notebook so that we can output the values of variables.

In [7]:
class MLTrainer:
    def __init__(
        self,
        model: Estimator,
        model_params: Dict,
        validator: Optional[Validator] = None,
        validation_params: Dict = {},
    ):
        self.model = model
        self.model_params = model_params
        self.validator = validator
        self.validation_params = validation_params

        # Provide by strategy if needed
        self.history = None
        self.horizon = None
        self.models: List[Estimator] = []
        self.scores: List[float] = []
        self.columns: List[str] = []

    def fit(self, data: dict, pipeline: Pipeline, val_data: Optional[dict] = None) -> "MLTrainer":
        X, y = pipeline.generate(data)
        
        print("X for training: ", X.shape)
        display(pd.DataFrame(X, columns=pipeline.output_features).head())
        print("y for training: ", y.shape)
        print(y[:5], "\n")

        if val_data:
            X_val, y_val = pipeline.generate(val_data)
        else:
            X_val, y_val = None, None

        # Initialize columns' order and reorder columns
        self.features_argsort = np.argsort(pipeline.output_features)
        X = X[:, self.features_argsort]

        for fold_i, (X_train, y_train, X_val, y_val) in enumerate(
            self.validator(**self.validation_params).get_split(X, y, X_val, y_val)
        ):
            model = self.model(self.model_params)
            model.fit_one_fold(X_train, y_train, X_val, y_val)
            self.models.append(model)
            self.scores.append(model.score)

            print(f"Fold {fold_i}. Score: {model.score}")

        print(f"Mean score: {np.mean(self.scores).round(4)}")
        print(f"Std: {np.std(self.scores).round(4)}")

    def predict(self, data: dict, pipeline: Pipeline) -> np.ndarray:
        X, _ = pipeline.generate(data)

        print("X for testing: ", X.shape)
        display(pd.DataFrame(X, columns=pipeline.output_features).head())

        # Reorder columns
        X = X[:, self.features_argsort]

        models_preds = [model.predict(X) for model in self.models]
        y_pred = np.mean(models_preds, axis=0)

        y_pred = y_pred.reshape(pipeline.y_original_shape)

        return y_pred

class RecursiveStrategy(Strategy):
    def __init__(
        self,
        horizon: int,
        history: int,
        trainer: Union[MLTrainer, DLTrainer],
        pipeline: Pipeline,
        step: int = 1,
        model_horizon: int = 1,
        reduced: bool = False,
    ):
        super().__init__(horizon, history, trainer, pipeline, step)
        self.model_horizon = model_horizon
        self.reduced = reduced
        self.strategy_name = "recursive"

    @timing_decorator
    def fit(
        self,
        dataset: TSDataset,
    ) -> "RecursiveStrategy":
        features_idx = index_slicer.create_idx_data(
            dataset.data,
            self.model_horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )

        target_idx = index_slicer.create_idx_target(
            dataset.data,
            self.model_horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )

        data = self.pipeline.create_data_dict_for_pipeline(dataset, features_idx, target_idx)
        data = self.pipeline.fit_transform(data, self.strategy_name)
        
        print("raw_ts_X_for_training: ", data["raw_ts_X"].shape)
        display(data["raw_ts_X"].head())

        val_dataset = self.trainer.validation_params.get("validation_data")

        if val_dataset:
            val_features_idx = index_slicer.create_idx_data(
                val_dataset.data,
                self.model_horizon,
                self.history,
                self.step,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            val_target_idx = index_slicer.create_idx_target(
                val_dataset.data,
                self.model_horizon,
                self.history,
                self.step,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            val_data = self.pipeline.create_data_dict_for_pipeline(
                val_dataset, val_features_idx, val_target_idx
            )
            val_data = self.pipeline.transform(val_data)
        else:
            val_data = None

        if isinstance(self.trainer, DLTrainer):
            if self.strategy_name == "FlatWideMIMOStrategy":
                self.trainer.horizon = 1
            else:
                self.trainer.horizon = self.model_horizon
            self.trainer.history = self.history

        current_trainer = deepcopy(self.trainer)

        # In Recursive strategy, we train the individual model
        if isinstance(current_trainer, DLTrainer):
            checkpoint_path = current_trainer.checkpoint_path
            pretrained_path = current_trainer.pretrained_path

            current_trainer.checkpoint_path /= "trainer_0"
            if pretrained_path:
                current_trainer.pretrained_path /= "trainer_0"

        current_trainer.fit(data, self.pipeline, val_data)

        if isinstance(current_trainer, DLTrainer):
            current_trainer.checkpoint_path = checkpoint_path
            current_trainer.pretrained_path = pretrained_path

        self.trainers.append(current_trainer)
        return self

    def make_step(self, step: int, dataset: TSDataset) -> TSDataset:
        test_idx = index_slicer.create_idx_test(
            dataset.data,
            self.horizon - step * self.model_horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )

        target_idx = index_slicer.create_idx_target(
            dataset.data,
            self.horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )[:, self.model_horizon * step : self.model_horizon * (step + 1)]

        data = self.pipeline.create_data_dict_for_pipeline(dataset, test_idx, target_idx)
        data = self.pipeline.transform(data)

        pred = self.trainers[0].predict(data, self.pipeline)
        pred = self.pipeline.inverse_transform_y(pred)

        dataset.data.loc[target_idx.reshape(-1), dataset.target_column] = pred.reshape(-1)

        return dataset

    @timing_decorator
    def predict(self, dataset: TSDataset, test_all: bool = False) -> pd.DataFrame:
        new_data = dataset.make_padded_test(
            self.horizon, self.history, test_all=test_all, step=self.step
        )
        new_dataset = TSDataset(new_data, dataset.columns_params, dataset.delta)

        if test_all:
            new_dataset.data = new_dataset.data.sort_values(
                [dataset.id_column, "segment_col", dataset.date_column]
            )

        if self.reduced:
            current_test_ids = index_slicer.create_idx_data(
                new_dataset.data,
                self.model_horizon,
                self.history,
                step=self.model_horizon,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            target_ids = index_slicer.create_idx_target(
                new_dataset.data,
                self.horizon,
                self.history,
                step=self.model_horizon,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            data = self.pipeline.create_data_dict_for_pipeline(
                new_dataset, current_test_ids, target_ids
            )
            data = self.pipeline.transform(data)

            print(f"raw_ts_X on recursive reduced", new_dataset.data[dataset.target_column].shape)
            display(new_dataset.data[new_dataset.data[dataset.id_column] == 0].tail(10))

            pred = self.trainers[0].predict(data, self.pipeline)
            pred = self.pipeline.inverse_transform_y(pred)

            new_dataset.data.loc[target_ids.reshape(-1), dataset.target_column] = pred.reshape(-1)

        else:
            for step in range(self.horizon // self.model_horizon):
                print(f"raw_ts_X on step: {step}", new_dataset.data[dataset.target_column].shape)
                display(new_dataset.data[new_dataset.data[dataset.id_column] == 0].tail(10))
                new_dataset = self.make_step(step, new_dataset)

        # Get dataframe with predictions only
        pred_df = self._make_preds_df(new_dataset, self.horizon, self.history)
        return pred_df

class MIMOStrategy(RecursiveStrategy):
    def __init__(
        self,
        horizon: int,
        history: int,
        trainer: Union[MLTrainer, DLTrainer],
        pipeline: Pipeline,
        step: int = 1,
    ):
        super().__init__(horizon, history, trainer, pipeline, step, model_horizon=horizon)
        self.strategy_name = "MIMOStrategy"

class DirectStrategy(RecursiveStrategy):
    def __init__(
        self,
        horizon: int,
        history: int,
        trainer: Union[MLTrainer, DLTrainer],
        pipeline: Pipeline,
        step: int = 1,
        model_horizon: int = 1,
        equal_train_size: bool = False,
    ):
        super().__init__(horizon, history, trainer, pipeline, step, model_horizon)
        self.equal_train_size = equal_train_size
        self.strategy_name = "direct"

    @timing_decorator
    def fit(
        self,
        dataset: TSDataset,
    ) -> "DirectStrategy":
        self.trainers = []

        if self.equal_train_size:
            features_idx = index_slicer.create_idx_data(
                dataset.data,
                self.horizon,
                self.history,
                self.step,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            target_idx = index_slicer.create_idx_target(
                dataset.data,
                self.horizon,
                self.history,
                self.step,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            data = self.pipeline.create_data_dict_for_pipeline(dataset, features_idx, target_idx)
            data = self.pipeline.fit_transform(data, self.strategy_name)

            val_dataset = self.trainer.validation_params.get("validation_data")

            if val_dataset:
                val_features_idx = index_slicer.create_idx_data(
                    val_dataset.data,
                    self.horizon,
                    self.history,
                    self.step,
                    date_column=val_dataset.date_column,
                    delta=val_dataset.delta,
                )

                val_target_idx = index_slicer.create_idx_target(
                    val_dataset.data,
                    self.horizon,
                    self.history,
                    self.step,
                    date_column=val_dataset.date_column,
                    delta=val_dataset.delta,
                )

                val_data = self.pipeline.create_data_dict_for_pipeline(
                    val_dataset, val_features_idx, val_target_idx
                )
                val_data = self.pipeline.transform(val_data)
            else:
                val_data = None

            for model_i, horizon in enumerate(range(1, self.horizon // self.model_horizon + 1)):
                target_idx = index_slicer.create_idx_target(
                    dataset.data,
                    self.horizon,
                    self.history,
                    self.step,
                    date_column=dataset.date_column,
                    delta=dataset.delta,
                )[:, (horizon - 1) * self.model_horizon : horizon * self.model_horizon]

                data["idx_y"] = target_idx

                if val_dataset:
                    val_target_idx = index_slicer.create_idx_target(
                        val_dataset.data,
                        self.horizon,
                        self.history,
                        self.step,
                        date_column=val_dataset.date_column,
                        delta=val_dataset.delta,
                    )[:, (horizon - 1) * self.model_horizon : horizon * self.model_horizon]

                    val_data["idx_y"] = val_target_idx

                if isinstance(self.trainer, DLTrainer):
                    self.trainer.horizon = self.model_horizon
                    self.trainer.history = self.history

                current_trainer = deepcopy(self.trainer)

                # In Direct strategy, we train several models, one for each model_horizon
                if isinstance(current_trainer, DLTrainer):
                    checkpoint_path = current_trainer.checkpoint_path
                    pretrained_path = current_trainer.pretrained_path

                    current_trainer.checkpoint_path /= f"trainer_{model_i}"
                    if pretrained_path:
                        current_trainer.pretrained_path /= f"trainer_{model_i}"

                print(f"Training model for horizon {horizon}".center(100, "-"))
                
                print("raw_ts_X_for_training: ", data["raw_ts_X"].shape)

                data["X"] = np.array([])
                data["y"] = np.array([])

                current_trainer.fit(data, self.pipeline, val_data)

                if isinstance(current_trainer, DLTrainer):
                    current_trainer.checkpoint_path = checkpoint_path
                    current_trainer.pretrained_path = pretrained_path

                self.trainers.append(current_trainer)

        else:
            for model_i, horizon in enumerate(range(1, self.horizon // self.model_horizon + 1)):
                features_idx = index_slicer.create_idx_data(
                    dataset.data,
                    self.model_horizon * horizon,
                    self.history,
                    self.step,
                    date_column=dataset.date_column,
                    delta=dataset.delta,
                )

                target_idx = index_slicer.create_idx_target(
                    dataset.data,
                    self.model_horizon * horizon,
                    self.history,
                    self.step,
                    date_column=dataset.date_column,
                    delta=dataset.delta,
                    n_last_horizon=self.model_horizon,
                )

                data = self.pipeline.create_data_dict_for_pipeline(
                    dataset, features_idx, target_idx
                )
                data = self.pipeline.fit_transform(data, self.strategy_name)

                val_dataset = self.trainer.validation_params.get("validation_data")

                if val_dataset:
                    val_features_idx = index_slicer.create_idx_data(
                        val_dataset.data,
                        self.model_horizon * horizon,
                        self.history,
                        self.step,
                        date_column=val_dataset.date_column,
                        delta=val_dataset.delta,
                    )

                    val_target_idx = index_slicer.create_idx_target(
                        val_dataset.data,
                        self.model_horizon * horizon,
                        self.history,
                        self.step,
                        date_column=val_dataset.date_column,
                        delta=val_dataset.delta,
                        n_last_horizon=self.model_horizon,
                    )

                    val_data = self.pipeline.create_data_dict_for_pipeline(
                        val_dataset, val_features_idx, val_target_idx
                    )
                    val_data = self.pipeline.transform(val_data)
                else:
                    val_data = None

                if isinstance(self.trainer, DLTrainer):
                    self.trainer.horizon = self.model_horizon
                    self.trainer.history = self.history

                current_trainer = deepcopy(self.trainer)

                # In Direct strategy, we train several models, one for each model_horizon
                if isinstance(current_trainer, DLTrainer):
                    checkpoint_path = current_trainer.checkpoint_path
                    pretrained_path = current_trainer.pretrained_path

                    current_trainer.checkpoint_path /= f"trainer_{model_i}"
                    if pretrained_path:
                        current_trainer.pretrained_path /= f"trainer_{model_i}"

                print(f"Training model for horizon {horizon}".center(100, "-"))
                
                print("raw_ts_X_for_training: ", data["raw_ts_X"].shape)

                current_trainer.fit(data, self.pipeline, val_data)

                if isinstance(current_trainer, DLTrainer):
                    current_trainer.checkpoint_path = checkpoint_path
                    current_trainer.pretrained_path = pretrained_path

                self.trainers.append(current_trainer)

        return self

    def make_step(self, step, dataset):
        test_idx = index_slicer.create_idx_test(
            dataset.data,
            self.horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )
        target_idx = index_slicer.create_idx_target(
            dataset.data,
            self.horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )[:, self.model_horizon * step : self.model_horizon * (step + 1)]

        data = self.pipeline.create_data_dict_for_pipeline(dataset, test_idx, target_idx)
        data = self.pipeline.transform(data)

        pred = self.trainers[step].predict(data, self.pipeline)
        pred = self.pipeline.inverse_transform_y(pred)

        dataset.data.loc[target_idx.reshape(-1), dataset.target_column] = pred.reshape(-1)

        return dataset

class FlatWideMIMOStrategy(MIMOStrategy):
    def __init__(
        self,
        horizon: int,
        history: int,
        trainer: Union[MLTrainer, DLTrainer],
        pipeline: Pipeline,
        step: int = 1,
    ):
        super().__init__(horizon, history, trainer, pipeline, step)
        self.strategy_name = "FlatWideMIMOStrategy"

# Loading Data

To make it easier to track and understand how features and targets are generated depending on the strategy, we will use a synthetic series with the following structure:
- 10 series, each with 1000 points.
- The value in the `value` column is represented as `{id - 1}{point}`, meaning it is a concatenation of the series ID and the time point number within that series. For example, the value `5234` corresponds to the 234th point in the series with `id=4`.

In [8]:
df = pd.read_csv("./datasets/global/simulated_data_to_check.csv")
display(df.iloc[:10])
display(df.iloc[2000:2010])

,value,date,id
0,1000,2020-01-01,0
1,1001,2020-01-02,0
2,1002,2020-01-03,0
3,1003,2020-01-04,0
4,1004,2020-01-05,0
5,1005,2020-01-06,0
6,1006,2020-01-07,0
7,1007,2020-01-08,0
8,1008,2020-01-09,0
9,1009,2020-01-10,0


,value,date,id
2000,3000,2020-01-01,2
2001,3001,2020-01-02,2
2002,3002,2020-01-03,2
2003,3003,2020-01-04,2
2004,3004,2020-01-05,2
2005,3005,2020-01-06,2
2006,3006,2020-01-07,2
2007,3007,2020-01-08,2
2008,3008,2020-01-09,2
2009,3009,2020-01-10,2


In [9]:
horizon = 4
history = 7

In [10]:
dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

dataset = TSDataset(
    data=df,
    columns_params=dataset_params,
    print_freq_period_info=True,
)

freq: Day; period: 1


In [11]:
pipeline_easy_params = {
    "target_lags": 7,
    "date_lags": 4,
}

# Configure the model parameters
model = CatBoost
model_params = {
    "loss_function": "MultiRMSE",
    "early_stopping_rounds": 100,
    "verbose": 500,
}

# Configure the validation parameters
validation = KFoldCrossValidator
validation_params = {
    "n_splits": 2,
}

trainer_params = {}
trainer = MLTrainer(
    model,
    model_params,
    validation,
    validation_params,
)

# Description of Strategies

## Multi-series prediction strategies:

We will examine how the feature and target matrices look later using the MIMO strategy as an example.

**Local-modelling**:
  - An individual model for each time series. 
  - Each time series as independent from others.

<img src="imgs/local.png" alt="local strategy" width="300"/>

**Global-modelling**:
  - A single model for all time series.
  - Features created from each series do not overlap with other series. Series are related but modeled separately.

<img src="imgs/global.png" alt="global strategy" width="300"/>

**Multivariate-modelling**:
  - A single model for all time series. 
  - Features created from each series are concatenated at each time step. Try to capture dependencies between the series at the same time point.

<img src="imgs/multivariate.png" alt="multivariate strategy" width="300"/>

## Multi-point-ahead prediction strategies:

Представим, что у нас есть временной ряд:

<img src="imgs/time_series.png" alt="time series" width="300"/>

Пусть на картинках у нас будет пример для следующих параметров (он не будет совпадать с примерами кода!):

$N=11$ (всего видим 11 точек)

$history=7$ (для предсказания следующей точки используем 7 предыдущих)

$horizon=3$

<img src="imgs/time_series_example.png" alt="time series example" width="300"/>

**MIMO (Multi-input-multi-output)**:
- One model is trained and used for the entire forecast horizon at once. 

<img src="imgs/mimo.png" alt="mimo" width="300"/>

- Note 1: This strategy can also accommodate exogenous features (for local- or global-modelling strategies). For more details on how to set exogenous features, refer to Tutorial 1.

__global-modelling__

In [7]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = MIMOStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
)

In [8]:
strategy.fit(dataset)

raw_ts_X_for_training:  (10000, 16)


,value,date,id,value__standard_scaler,date__season_y,date__season_m,date__season_d,date__season_doy,date__season_wd,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,id__label,id__label__standard_scaler
0,1000.0,2020-01-01,0,-1.729454,2020,1,5,5,6,-1.150365,-1.557466,-1.228139,-1.651456,1.500926,0,-1.566621
1,1001.0,2020-01-02,0,-1.725992,2020,1,6,6,0,-1.150365,-1.557466,-1.114169,-1.641528,-1.498926,0,-1.566621
2,1002.0,2020-01-03,0,-1.722530,2020,1,7,7,1,-1.150365,-1.557466,-1.000199,-1.631600,-0.998951,0,-1.566621
3,1003.0,2020-01-04,0,-1.719067,2020,1,8,8,2,-1.150365,-1.557466,-0.886229,-1.621673,-0.498975,0,-1.566621
4,1004.0,2020-01-05,0,-1.715605,2020,1,9,9,3,-1.150365,-1.557466,-0.772259,-1.611745,0.001000,0,-1.566621


X for training:  (9900, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.150365,-1.150365,-1.150365,...,-0.544320,-1.621673,-1.611745,-1.601817,-1.591889,-0.498975,0.001000,0.500975,1.000950,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.150365,-1.150365,-1.150365,...,-0.430350,-1.611745,-1.601817,-1.591889,-1.581962,0.001000,0.500975,1.000950,1.500926,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.150365,-1.150365,-1.150365,...,-0.316380,-1.601817,-1.591889,-1.581962,-1.572034,0.500975,1.000950,1.500926,-1.498926,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.150365,-1.150365,-1.150365,...,-0.202410,-1.591889,-1.581962,-1.572034,-1.562106,1.000950,1.500926,-1.498926,-0.998951,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.150365,-1.150365,-1.150365,...,-0.088441,-1.581962,-1.572034,-1.562106,-1.552178,1.500926,-1.498926,-0.998951,-0.498975,-1.566621


y for training:  (9900, 4)
[[-1.70521765 -1.70175528 -1.69829291 -1.69483054]
 [-1.70175528 -1.69829291 -1.69483054 -1.69136817]
 [-1.69829291 -1.69483054 -1.69136817 -1.6879058 ]
 [-1.69483054 -1.69136817 -1.6879058  -1.68444342]
 [-1.69136817 -1.6879058  -1.68444342 -1.68098105]] 

0:	learn: 1.9179675	test: 1.9252979	best: 1.9252979 (0)	total: 58.9ms	remaining: 58.8s
500:	learn: 0.0071597	test: 0.0072534	best: 0.0072534 (500)	total: 1.59s	remaining: 1.58s
999:	learn: 0.0039772	test: 0.0042419	best: 0.0042419 (999)	total: 2.95s	remaining: 0us

bestTest = 0.004241889806
bestIteration = 999

Fold 0. Score: 0.004241889806475644
0:	learn: 1.9239154	test: 1.9197469	best: 1.9197469 (0)	total: 3.04ms	remaining: 3.03s
500:	learn: 0.0072229	test: 0.0074385	best: 0.0074385 (500)	total: 1.35s	remaining: 1.34s
999:	learn: 0.0040616	test: 0.0043572	best: 0.0043572 (999)	total: 2.53s	remaining: 0us

bestTest = 0.004357247947
bestIteration = 999

Fold 1. Score: 0.004357247946880891
Mean score: 0.004

(5.744672060012817, <__main__.MIMOStrategy at 0x310fdf9a0>)

In [9]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on step: 0 (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.174069


In [10]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1988.156501
1,0,2022-09-28,1989.156504
2,0,2022-09-29,1990.156503
3,0,2022-09-30,1991.156504
4,1,2022-09-27,2988.130075
5,1,2022-09-28,2989.130078
6,1,2022-09-29,2990.130077
7,1,2022-09-30,2991.130078
8,2,2022-09-27,3988.124939
9,2,2022-09-28,3989.124942


__multivariate-modelling__

In [11]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=True)

strategy = MIMOStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
)

In [12]:
strategy.fit(dataset)

raw_ts_X_for_training:  (10000, 16)


,value,date,id,value__standard_scaler,date__season_y,date__season_m,date__season_d,date__season_doy,date__season_wd,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,id__label,id__label__standard_scaler
0,1000.0,2020-01-01,0,-1.729454,2020,1,5,5,6,-1.150365,-1.557466,-1.228139,-1.651456,1.500926,0,-1.566621
1,1001.0,2020-01-02,0,-1.725992,2020,1,6,6,0,-1.150365,-1.557466,-1.114169,-1.641528,-1.498926,0,-1.566621
2,1002.0,2020-01-03,0,-1.722530,2020,1,7,7,1,-1.150365,-1.557466,-1.000199,-1.631600,-0.998951,0,-1.566621
3,1003.0,2020-01-04,0,-1.719067,2020,1,8,8,2,-1.150365,-1.557466,-0.886229,-1.621673,-0.498975,0,-1.566621
4,1004.0,2020-01-05,0,-1.715605,2020,1,9,9,3,-1.150365,-1.557466,-0.772259,-1.611745,0.001000,0,-1.566621


X for training:  (990, 90)


,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,date__season_y__standard_scaler__lag_0,date__season_m__standard_scaler__lag_3,date__season_m__standard_scaler__lag_2,date__season_m__standard_scaler__lag_1,date__season_m__standard_scaler__lag_0,date__season_d__standard_scaler__lag_3,date__season_d__standard_scaler__lag_2,...,value__standard_scaler__lag_2__8,value__standard_scaler__lag_1__8,value__standard_scaler__lag_0__8,value__standard_scaler__lag_6__9,value__standard_scaler__lag_5__9,value__standard_scaler__lag_4__9,value__standard_scaler__lag_3__9,value__standard_scaler__lag_2__9,value__standard_scaler__lag_1__9,value__standard_scaler__lag_0__9
0,-1.150365,-1.150365,-1.150365,-1.150365,-1.557466,-1.557466,-1.557466,-1.557466,-0.886229,-0.772259,...,-1.715605,-1.712142,-1.708680,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680
1,-1.150365,-1.150365,-1.150365,-1.150365,-1.557466,-1.557466,-1.557466,-1.557466,-0.772259,-0.658290,...,-1.712142,-1.708680,-1.705218,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218
2,-1.150365,-1.150365,-1.150365,-1.150365,-1.557466,-1.557466,-1.557466,-1.557466,-0.658290,-0.544320,...,-1.708680,-1.705218,-1.701755,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755
3,-1.150365,-1.150365,-1.150365,-1.150365,-1.557466,-1.557466,-1.557466,-1.557466,-0.544320,-0.430350,...,-1.705218,-1.701755,-1.698293,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293
4,-1.150365,-1.150365,-1.150365,-1.150365,-1.557466,-1.557466,-1.557466,-1.557466,-0.430350,-0.316380,...,-1.701755,-1.698293,-1.694831,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831


y for training:  (990, 40)
[[-1.70521765 -1.70175528 -1.69829291 -1.69483054 -1.70521765 -1.70175528
  -1.69829291 -1.69483054 -1.70521765 -1.70175528 -1.69829291 -1.69483054
  -1.70521765 -1.70175528 -1.69829291 -1.69483054 -1.70521765 -1.70175528
  -1.69829291 -1.69483054 -1.70521765 -1.70175528 -1.69829291 -1.69483054
  -1.70521765 -1.70175528 -1.69829291 -1.69483054 -1.70521765 -1.70175528
  -1.69829291 -1.69483054 -1.70521765 -1.70175528 -1.69829291 -1.69483054
  -1.70521765 -1.70175528 -1.69829291 -1.69483054]
 [-1.70175528 -1.69829291 -1.69483054 -1.69136817 -1.70175528 -1.69829291
  -1.69483054 -1.69136817 -1.70175528 -1.69829291 -1.69483054 -1.69136817
  -1.70175528 -1.69829291 -1.69483054 -1.69136817 -1.70175528 -1.69829291
  -1.69483054 -1.69136817 -1.70175528 -1.69829291 -1.69483054 -1.69136817
  -1.70175528 -1.69829291 -1.69483054 -1.69136817 -1.70175528 -1.69829291
  -1.69483054 -1.69136817 -1.70175528 -1.69829291 -1.69483054 -1.69136817
  -1.70175528 -1.69829291 -1.69483

(137.88563323020935, <__main__.MIMOStrategy at 0x310fdfc10>)

In [13]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on step: 0 (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (1, 90)


,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,date__season_y__standard_scaler__lag_0,date__season_m__standard_scaler__lag_3,date__season_m__standard_scaler__lag_2,date__season_m__standard_scaler__lag_1,date__season_m__standard_scaler__lag_0,date__season_d__standard_scaler__lag_3,date__season_d__standard_scaler__lag_2,...,value__standard_scaler__lag_2__8,value__standard_scaler__lag_1__8,value__standard_scaler__lag_0__8,value__standard_scaler__lag_6__9,value__standard_scaler__lag_5__9,value__standard_scaler__lag_4__9,value__standard_scaler__lag_3__9,value__standard_scaler__lag_2__9,value__standard_scaler__lag_1__9,value__standard_scaler__lag_0__9
0,1.375134,1.375134,1.375134,1.375134,0.870384,0.870384,0.870384,0.870384,1.279197,1.393167,...,1.72253,1.725992,1.729454,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454


In [14]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1991.03783
1,0,2022-09-28,1992.037829
2,0,2022-09-29,1993.037832
3,0,2022-09-30,1994.037829
4,1,2022-09-27,2991.03783
5,1,2022-09-28,2992.037829
6,1,2022-09-29,2993.037832
7,1,2022-09-30,2994.037829
8,2,2022-09-27,3991.03783
9,2,2022-09-28,3992.037829


We can see that when using multivariate modeling, the features and targets of individual series are concatenated.

__local-modelling__

In fact, there is no independent `local-modelling` strategy in the library. However, it can be achieved by iterating over each time series `id` separately in a loop and then concatenating the results.

In [15]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = MIMOStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
)

preds_list = []
for ts_id in dataset.data["id"].unique():
    print(f"Training for ts_id: {ts_id}".center(100, "-"), "\n")
    
    current_df = df[df["id"] == ts_id]
    current_dataset = TSDataset(
        data=current_df,
        columns_params=dataset_params,
        print_freq_period_info=True,
    )

    strategy.fit(current_dataset)
    forecast_time, current_pred = strategy.predict(current_dataset)
    preds_list.append(current_pred)

---------------------------------------Training for ts_id: 0---------------------------------------- 

freq: Day; period: 1
raw_ts_X_for_training:  (1000, 16)


,value,date,id,value__standard_scaler,date__season_y,date__season_m,date__season_d,date__season_doy,date__season_wd,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,id__label,id__label__standard_scaler
0,1000.0,2020-01-01,0,-1.729454,2020,1,5,5,6,-1.149847,-1.556765,-1.227586,-1.650713,1.500250,0,NaN
1,1001.0,2020-01-02,0,-1.725992,2020,1,6,6,0,-1.149847,-1.556765,-1.113667,-1.640789,-1.498251,0,NaN
2,1002.0,2020-01-03,0,-1.722530,2020,1,7,7,1,-1.149847,-1.556765,-0.999749,-1.630866,-0.998501,0,NaN
3,1003.0,2020-01-04,0,-1.719067,2020,1,8,8,2,-1.149847,-1.556765,-0.885830,-1.620943,-0.498751,0,NaN
4,1004.0,2020-01-05,0,-1.715605,2020,1,9,9,3,-1.149847,-1.556765,-0.771912,-1.611019,0.001000,0,NaN


X for training:  (990, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.149847,-1.149847,-1.149847,...,-0.544075,-1.620943,-1.611019,-1.601096,-1.591173,-0.498751,0.001000,0.500750,1.000500,NaN
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.149847,-1.149847,-1.149847,...,-0.430156,-1.611019,-1.601096,-1.591173,-1.581249,0.001000,0.500750,1.000500,1.500250,NaN
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.149847,-1.149847,-1.149847,...,-0.316238,-1.601096,-1.591173,-1.581249,-1.571326,0.500750,1.000500,1.500250,-1.498251,NaN
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.149847,-1.149847,-1.149847,...,-0.202319,-1.591173,-1.581249,-1.571326,-1.561403,1.000500,1.500250,-1.498251,-0.998501,NaN
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.149847,-1.149847,-1.149847,...,-0.088401,-1.581249,-1.571326,-1.561403,-1.551480,1.500250,-1.498251,-0.998501,-0.498751,NaN


y for training:  (990, 4)
[[-1.70521765 -1.70175528 -1.69829291 -1.69483054]
 [-1.70175528 -1.69829291 -1.69483054 -1.69136817]
 [-1.69829291 -1.69483054 -1.69136817 -1.6879058 ]
 [-1.69483054 -1.69136817 -1.6879058  -1.68444342]
 [-1.69136817 -1.6879058  -1.68444342 -1.68098105]] 

0:	learn: 1.9332358	test: 1.9222850	best: 1.9222850 (0)	total: 5.22ms	remaining: 5.21s
500:	learn: 0.0135313	test: 0.0312388	best: 0.0312388 (500)	total: 802ms	remaining: 798ms
999:	learn: 0.0074311	test: 0.0291400	best: 0.0291400 (999)	total: 1.61s	remaining: 0us

bestTest = 0.02914004013
bestIteration = 999

Fold 0. Score: 0.029140040129593817
0:	learn: 1.9164336	test: 1.9384335	best: 1.9384335 (0)	total: 1.64ms	remaining: 1.63s
500:	learn: 0.0132088	test: 0.0234053	best: 0.0234053 (500)	total: 772ms	remaining: 769ms
999:	learn: 0.0069993	test: 0.0205025	best: 0.0205012 (998)	total: 1.66s	remaining: 0us

bestTest = 0.02050117144
bestIteration = 998

Shrink model to first 999 iterations.
Fold 1. Score: 0.0

...

,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.149847,-1.149847,-1.149847,...,-0.544075,-1.620943,-1.611019,-1.601096,-1.591173,-0.498751,0.001000,0.500750,1.000500,NaN
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.149847,-1.149847,-1.149847,...,-0.430156,-1.611019,-1.601096,-1.591173,-1.581249,0.001000,0.500750,1.000500,1.500250,NaN
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.149847,-1.149847,-1.149847,...,-0.316238,-1.601096,-1.591173,-1.581249,-1.571326,0.500750,1.000500,1.500250,-1.498251,NaN
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.149847,-1.149847,-1.149847,...,-0.202319,-1.591173,-1.581249,-1.571326,-1.561403,1.000500,1.500250,-1.498251,-0.998501,NaN
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.149847,-1.149847,-1.149847,...,-0.088401,-1.581249,-1.571326,-1.561403,-1.551480,1.500250,-1.498251,-0.998501,-0.498751,NaN


y for training:  (990, 4)
[[-1.70521765 -1.70175528 -1.69829291 -1.69483054]
 [-1.70175528 -1.69829291 -1.69483054 -1.69136817]
 [-1.69829291 -1.69483054 -1.69136817 -1.6879058 ]
 [-1.69483054 -1.69136817 -1.6879058  -1.68444342]
 [-1.69136817 -1.6879058  -1.68444342 -1.68098105]] 

0:	learn: 1.9332358	test: 1.9222850	best: 1.9222850 (0)	total: 2.97ms	remaining: 2.96s
500:	learn: 0.0135313	test: 0.0312388	best: 0.0312388 (500)	total: 742ms	remaining: 739ms
999:	learn: 0.0074311	test: 0.0291400	best: 0.0291400 (999)	total: 1.49s	remaining: 0us

bestTest = 0.02914004013
bestIteration = 999

Fold 0. Score: 0.029140040129593817
0:	learn: 1.9164336	test: 1.9384335	best: 1.9384335 (0)	total: 1.72ms	remaining: 1.71s
500:	learn: 0.0132088	test: 0.0234053	best: 0.0234053 (500)	total: 767ms	remaining: 764ms
999:	learn: 0.0069993	test: 0.0205025	best: 0.0205012 (998)	total: 1.51s	remaining: 0us

bestTest = 0.02050117144
bestIteration = 998

Shrink model to first 999 iterations.
Fold 1. Score: 0.0

,value,date,id


X for testing:  (1, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.374515,1.374515,1.374515,...,1.620377,0.978964,0.988887,0.998811,1.008734,-0.998501,-0.498751,0.001,0.50075,NaN


In [16]:
pd.concat(preds_list).head(10)

,id,date,value
0,0,2022-09-27,1982.81454
1,0,2022-09-28,1983.814539
2,0,2022-09-29,1984.814543
3,0,2022-09-30,1985.814539
0,1,2022-09-27,2982.81454
1,1,2022-09-28,2983.814539
2,1,2022-09-29,2984.814543
3,1,2022-09-30,2985.814539
0,2,2022-09-27,3982.81454
1,2,2022-09-28,3983.814539


**Recursive**:
- One model is used for the entire forecast horizon. 
- training: The model is trained to predict one point ahead.
- prediction: The model iteratively predicts each point, using previous predictions to update the features in the test data.

<img src="imgs/recursive.png" alt="recursive" width="300"/>

In [17]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = RecursiveStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
)

In [18]:
strategy.fit(dataset)

raw_ts_X_for_training:  (10000, 16)


,value,date,id,value__standard_scaler,date__season_y,date__season_m,date__season_d,date__season_doy,date__season_wd,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,id__label,id__label__standard_scaler
0,1000.0,2020-01-01,0,-1.729454,2020,1,2,2,3,-1.143795,-1.546358,-1.561575,-1.668794,-0.000500,0,-1.566621
1,1001.0,2020-01-02,0,-1.725992,2020,1,3,3,4,-1.143795,-1.546358,-1.447574,-1.658893,0.499288,0,-1.566621
2,1002.0,2020-01-03,0,-1.722530,2020,1,4,4,5,-1.143795,-1.546358,-1.333574,-1.648991,0.999076,0,-1.566621
3,1003.0,2020-01-04,0,-1.719067,2020,1,5,5,6,-1.143795,-1.546358,-1.219574,-1.639090,1.498863,0,-1.566621
4,1004.0,2020-01-05,0,-1.715605,2020,1,6,6,0,-1.143795,-1.546358,-1.105574,-1.629188,-1.499863,0,-1.566621


X for training:  (9930, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.143795,-1.143795,-1.143795,...,-0.877573,-1.639090,-1.629188,-1.619287,-1.609385,1.498863,-1.499863,-1.000075,-0.500287,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.143795,-1.143795,-1.143795,...,-0.763573,-1.629188,-1.619287,-1.609385,-1.599484,-1.499863,-1.000075,-0.500287,-0.000500,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.143795,-1.143795,-1.143795,...,-0.649573,-1.619287,-1.609385,-1.599484,-1.589582,-1.000075,-0.500287,-0.000500,0.499288,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.143795,-1.143795,-1.143795,...,-0.535573,-1.609385,-1.599484,-1.589582,-1.579681,-0.500287,-0.000500,0.499288,0.999076,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.143795,-1.143795,-1.143795,...,-0.421573,-1.599484,-1.589582,-1.579681,-1.569779,-0.000500,0.499288,0.999076,1.498863,-1.566621


y for training:  (9930, 1)
[[-1.70521765]
 [-1.70175528]
 [-1.69829291]
 [-1.69483054]
 [-1.69136817]] 

0:	learn: 0.9606043	test: 0.9668573	best: 0.9668573 (0)	total: 1.39ms	remaining: 1.38s
500:	learn: 0.0037506	test: 0.0038327	best: 0.0038327 (500)	total: 600ms	remaining: 597ms
999:	learn: 0.0021159	test: 0.0022825	best: 0.0022825 (999)	total: 1.19s	remaining: 0us

bestTest = 0.002282486309
bestIteration = 999

Fold 0. Score: 0.002282486308604412
0:	learn: 0.9660711	test: 0.9614294	best: 0.9614294 (0)	total: 1.21ms	remaining: 1.21s
500:	learn: 0.0038068	test: 0.0039266	best: 0.0039266 (500)	total: 588ms	remaining: 585ms
999:	learn: 0.0020537	test: 0.0022117	best: 0.0022117 (999)	total: 1.18s	remaining: 0us

bestTest = 0.002211714388
bestIteration = 999

Fold 1. Score: 0.002211714388200288
Mean score: 0.0022
Std: 0.0


(2.5806527137756348, <__main__.RecursiveStrategy at 0x311004610>)

In [19]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on step: 0 (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.383929,1.383929,1.383929,...,1.28843,0.955096,0.964997,0.974899,0.9848,0.999076,1.498863,-1.499863,-1.000075,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.383929,1.383929,1.383929,...,1.28843,0.955096,0.964997,0.974899,0.9848,0.999076,1.498863,-1.499863,-1.000075,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.383929,1.383929,1.383929,...,1.28843,0.955096,0.964997,0.974899,0.9848,0.999076,1.498863,-1.499863,-1.000075,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.383929,1.383929,1.383929,...,1.28843,0.955096,0.964997,0.974899,0.9848,0.999076,1.498863,-1.499863,-1.000075,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.383929,1.383929,1.383929,...,1.28843,0.955096,0.964997,0.974899,0.9848,0.999076,1.498863,-1.499863,-1.000075,-0.174069


raw_ts_X on step: 1 (110,)


...

,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.715605,1.719067,1.72253,1.725992,1.729454,1.720657,1.713092,1.383929,1.383929,1.383929,...,1.516431,0.974899,0.9848,0.994702,1.004603,-1.499863,-1.000075,-0.500287,-0.0005,-1.566621
1,1.715605,1.719067,1.72253,1.725992,1.729454,1.720664,1.713099,1.383929,1.383929,1.383929,...,1.516431,0.974899,0.9848,0.994702,1.004603,-1.499863,-1.000075,-0.500287,-0.0005,-1.218483
2,1.715605,1.719067,1.72253,1.725992,1.729454,1.720759,1.713194,1.383929,1.383929,1.383929,...,1.516431,0.974899,0.9848,0.994702,1.004603,-1.499863,-1.000075,-0.500287,-0.0005,-0.870345
3,1.715605,1.719067,1.72253,1.725992,1.729454,1.721067,1.713506,1.383929,1.383929,1.383929,...,1.516431,0.974899,0.9848,0.994702,1.004603,-1.499863,-1.000075,-0.500287,-0.0005,-0.522207
4,1.715605,1.719067,1.72253,1.725992,1.729454,1.721081,1.713520,1.383929,1.383929,1.383929,...,1.516431,0.974899,0.9848,0.994702,1.004603,-1.499863,-1.000075,-0.500287,-0.0005,-0.174069


raw_ts_X on step: 3 (110,)


,value,date,id
1,1994.000000,2022-09-21,0
2,1995.000000,2022-09-22,0
3,1996.000000,2022-09-23,0
4,1997.000000,2022-09-24,0
5,1998.000000,2022-09-25,0
6,1999.000000,2022-09-26,0
7,1996.459137,2022-09-27,0
8,1994.274164,2022-09-28,0
9,1993.528653,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.719067,1.72253,1.725992,1.729454,1.720657,1.713092,1.710510,1.383929,1.383929,1.383929,...,1.630431,0.9848,0.994702,1.004603,1.014505,-1.000075,-0.500287,-0.0005,0.499288,-1.566621
1,1.719067,1.72253,1.725992,1.729454,1.720664,1.713099,1.710518,1.383929,1.383929,1.383929,...,1.630431,0.9848,0.994702,1.004603,1.014505,-1.000075,-0.500287,-0.0005,0.499288,-1.218483
2,1.719067,1.72253,1.725992,1.729454,1.720759,1.713194,1.710613,1.383929,1.383929,1.383929,...,1.630431,0.9848,0.994702,1.004603,1.014505,-1.000075,-0.500287,-0.0005,0.499288,-0.870345
3,1.719067,1.72253,1.725992,1.729454,1.721067,1.713506,1.710925,1.383929,1.383929,1.383929,...,1.630431,0.9848,0.994702,1.004603,1.014505,-1.000075,-0.500287,-0.0005,0.499288,-0.522207
4,1.719067,1.72253,1.725992,1.729454,1.721081,1.713520,1.710939,1.383929,1.383929,1.383929,...,1.630431,0.9848,0.994702,1.004603,1.014505,-1.000075,-0.500287,-0.0005,0.499288,-0.174069


In [20]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1996.459137
1,0,2022-09-28,1994.274164
2,0,2022-09-29,1993.528653
3,0,2022-09-30,1992.581575
4,1,2022-09-27,2996.461299
5,1,2022-09-28,2994.276326
6,1,2022-09-29,2993.530815
7,1,2022-09-30,2992.583707
8,2,2022-09-27,3996.488771
9,2,2022-09-28,3994.303798


You can observe how `raw_ts_X` is progressively filled at each step.

**- Note 1: There is an option to use a “reduced” version, where features are generated for all test observations at once, and unavailable values are filled with NaN.**

In [21]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = RecursiveStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
    reduced=True,
)

In [22]:
strategy.fit(dataset)

raw_ts_X_for_training:  (10000, 16)


,value,date,id,value__standard_scaler,date__season_y,date__season_m,date__season_d,date__season_doy,date__season_wd,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,id__label,id__label__standard_scaler
0,1000.0,2020-01-01,0,-1.729454,2020,1,2,2,3,-1.143795,-1.546358,-1.561575,-1.668794,-0.000500,0,-1.566621
1,1001.0,2020-01-02,0,-1.725992,2020,1,3,3,4,-1.143795,-1.546358,-1.447574,-1.658893,0.499288,0,-1.566621
2,1002.0,2020-01-03,0,-1.722530,2020,1,4,4,5,-1.143795,-1.546358,-1.333574,-1.648991,0.999076,0,-1.566621
3,1003.0,2020-01-04,0,-1.719067,2020,1,5,5,6,-1.143795,-1.546358,-1.219574,-1.639090,1.498863,0,-1.566621
4,1004.0,2020-01-05,0,-1.715605,2020,1,6,6,0,-1.143795,-1.546358,-1.105574,-1.629188,-1.499863,0,-1.566621


X for training:  (9930, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.143795,-1.143795,-1.143795,...,-0.877573,-1.639090,-1.629188,-1.619287,-1.609385,1.498863,-1.499863,-1.000075,-0.500287,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.143795,-1.143795,-1.143795,...,-0.763573,-1.629188,-1.619287,-1.609385,-1.599484,-1.499863,-1.000075,-0.500287,-0.000500,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.143795,-1.143795,-1.143795,...,-0.649573,-1.619287,-1.609385,-1.599484,-1.589582,-1.000075,-0.500287,-0.000500,0.499288,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.143795,-1.143795,-1.143795,...,-0.535573,-1.609385,-1.599484,-1.589582,-1.579681,-0.500287,-0.000500,0.499288,0.999076,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.143795,-1.143795,-1.143795,...,-0.421573,-1.599484,-1.589582,-1.579681,-1.569779,-0.000500,0.499288,0.999076,1.498863,-1.566621


y for training:  (9930, 1)
[[-1.70521765]
 [-1.70175528]
 [-1.69829291]
 [-1.69483054]
 [-1.69136817]] 

0:	learn: 0.9606043	test: 0.9668573	best: 0.9668573 (0)	total: 1.17ms	remaining: 1.17s
500:	learn: 0.0037506	test: 0.0038327	best: 0.0038327 (500)	total: 598ms	remaining: 596ms
999:	learn: 0.0021159	test: 0.0022825	best: 0.0022825 (999)	total: 1.21s	remaining: 0us

bestTest = 0.002282486309
bestIteration = 999

Fold 0. Score: 0.002282486308604412
0:	learn: 0.9660711	test: 0.9614294	best: 0.9614294 (0)	total: 1.13ms	remaining: 1.12s
500:	learn: 0.0038068	test: 0.0039266	best: 0.0039266 (500)	total: 610ms	remaining: 608ms
999:	learn: 0.0020537	test: 0.0022117	best: 0.0022117 (999)	total: 1.21s	remaining: 0us

bestTest = 0.002211714388
bestIteration = 999

Fold 1. Score: 0.002211714388200288
Mean score: 0.0022
Std: 0.0


(2.6219847202301025, <__main__.RecursiveStrategy at 0x311004d60>)

In [23]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on recursive reduced (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (40, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.708680,1.712142,1.715605,1.719067,1.722530,1.725992,1.729454,1.383929,1.383929,1.383929,...,1.630431,0.984800,0.994702,1.004603,1.014505,-1.000075,-0.500287,-0.000500,0.499288,-1.566621
1,1.712142,1.715605,1.719067,1.722530,1.725992,1.729454,NaN,1.383929,1.383929,1.383929,...,-1.675575,0.994702,1.004603,1.014505,1.024406,-0.500287,-0.000500,0.499288,0.999076,-1.566621
2,1.715605,1.719067,1.722530,1.725992,1.729454,NaN,NaN,1.383929,1.383929,1.383929,...,-1.561575,1.004603,1.014505,1.024406,1.034308,-0.000500,0.499288,0.999076,1.498863,-1.566621
3,1.719067,1.722530,1.725992,1.729454,NaN,NaN,NaN,1.383929,1.383929,1.383929,...,-1.447574,1.014505,1.024406,1.034308,1.044209,0.499288,0.999076,1.498863,-1.499863,-1.566621
4,1.708680,1.712142,1.715605,1.719067,1.722530,1.725992,1.729454,1.383929,1.383929,1.383929,...,1.630431,0.984800,0.994702,1.004603,1.014505,-1.000075,-0.500287,-0.000500,0.499288,-1.218483


In [24]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1992.750094
1,0,2022-09-28,1826.62222
2,0,2022-09-29,1708.359827
3,0,2022-09-30,1629.943595
4,1,2022-09-27,2992.752225
5,1,2022-09-28,2826.61879
6,1,2022-09-29,2708.356397
7,1,2022-09-30,2629.936806
8,2,2022-09-27,3992.778393
9,2,2022-09-28,3826.647477


**- Note 2: Recursive can also be combined with the MIMO strategy, allowing the model to predict model_horizon points ahead at each step. The overall forecasting horizon should be divisible by the horizon of the individual model without a remainder.**

In [25]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = RecursiveStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
    model_horizon=2,
)

In [26]:
strategy.fit(dataset)

raw_ts_X_for_training:  (10000, 16)


,value,date,id,value__standard_scaler,date__season_y,date__season_m,date__season_d,date__season_doy,date__season_wd,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,id__label,id__label__standard_scaler
0,1000.0,2020-01-01,0,-1.729454,2020,1,3,3,4,-1.145979,-1.550044,-1.450887,-1.663056,0.499725,0,-1.566621
1,1001.0,2020-01-02,0,-1.725992,2020,1,4,4,5,-1.145979,-1.550044,-1.336860,-1.653145,0.999450,0,-1.566621
2,1002.0,2020-01-03,0,-1.722530,2020,1,5,5,6,-1.145979,-1.550044,-1.222832,-1.643235,1.499176,0,-1.566621
3,1003.0,2020-01-04,0,-1.719067,2020,1,6,6,0,-1.145979,-1.550044,-1.108804,-1.633324,-1.499176,0,-1.566621
4,1004.0,2020-01-05,0,-1.715605,2020,1,7,7,1,-1.145979,-1.550044,-0.994777,-1.623414,-0.999450,0,-1.566621


X for training:  (9920, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.145979,-1.145979,-1.145979,...,-0.766722,-1.633324,-1.623414,-1.613504,-1.603593,-1.499176,-0.999450,-0.499725,0.000000,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.145979,-1.145979,-1.145979,...,-0.652694,-1.623414,-1.613504,-1.603593,-1.593683,-0.999450,-0.499725,0.000000,0.499725,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.145979,-1.145979,-1.145979,...,-0.538666,-1.613504,-1.603593,-1.593683,-1.583772,-0.499725,0.000000,0.499725,0.999450,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.145979,-1.145979,-1.145979,...,-0.424639,-1.603593,-1.593683,-1.583772,-1.573862,0.000000,0.499725,0.999450,1.499176,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.145979,-1.145979,-1.145979,...,-0.310611,-1.593683,-1.583772,-1.573862,-1.563952,0.499725,0.999450,1.499176,-1.499176,-1.566621


y for training:  (9920, 2)
[[-1.70521765 -1.70175528]
 [-1.70175528 -1.69829291]
 [-1.69829291 -1.69483054]
 [-1.69483054 -1.69136817]
 [-1.69136817 -1.6879058 ]] 

0:	learn: 1.3609687	test: 1.3619154	best: 1.3619154 (0)	total: 1.94ms	remaining: 1.93s
500:	learn: 0.0052110	test: 0.0055076	best: 0.0055076 (500)	total: 771ms	remaining: 768ms
999:	learn: 0.0029246	test: 0.0032282	best: 0.0032282 (999)	total: 1.55s	remaining: 0us

bestTest = 0.003228249025
bestIteration = 999

Fold 0. Score: 0.0032282490248902312
0:	learn: 1.3611017	test: 1.3621758	best: 1.3621758 (0)	total: 1.64ms	remaining: 1.64s
500:	learn: 0.0049621	test: 0.0052201	best: 0.0052201 (500)	total: 777ms	remaining: 774ms
999:	learn: 0.0027790	test: 0.0030534	best: 0.0030534 (999)	total: 1.54s	remaining: 0us

bestTest = 0.00305343819
bestIteration = 999

Fold 1. Score: 0.0030534381902755756
Mean score: 0.0031
Std: 0.0001


(3.2906980514526367, <__main__.RecursiveStrategy at 0x311038a60>)

In [27]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on step: 0 (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.380987,1.380987,1.380987,...,1.399803,0.963202,0.973113,0.983023,0.992934,1.499176,-1.499176,-0.99945,-0.499725,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.380987,1.380987,1.380987,...,1.399803,0.963202,0.973113,0.983023,0.992934,1.499176,-1.499176,-0.99945,-0.499725,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.380987,1.380987,1.380987,...,1.399803,0.963202,0.973113,0.983023,0.992934,1.499176,-1.499176,-0.99945,-0.499725,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.380987,1.380987,1.380987,...,1.399803,0.963202,0.973113,0.983023,0.992934,1.499176,-1.499176,-0.99945,-0.499725,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.380987,1.380987,1.380987,...,1.399803,0.963202,0.973113,0.983023,0.992934,1.499176,-1.499176,-0.99945,-0.499725,-0.174069


raw_ts_X on step: 1 (110,)


,value,date,id
1,1994.000000,2022-09-21,0
2,1995.000000,2022-09-22,0
3,1996.000000,2022-09-23,0
4,1997.000000,2022-09-24,0
5,1998.000000,2022-09-25,0
6,1999.000000,2022-09-26,0
7,1993.363236,2022-09-27,0
8,1994.363237,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.715605,1.719067,1.72253,1.725992,1.729454,1.709938,1.713400,1.380987,1.380987,1.380987,...,1.627858,0.983023,0.992934,1.002844,1.012755,-0.99945,-0.499725,0.0,0.499725,-1.566621
1,1.715605,1.719067,1.72253,1.725992,1.729454,1.709947,1.713409,1.380987,1.380987,1.380987,...,1.627858,0.983023,0.992934,1.002844,1.012755,-0.99945,-0.499725,0.0,0.499725,-1.218483
2,1.715605,1.719067,1.72253,1.725992,1.729454,1.709943,1.713405,1.380987,1.380987,1.380987,...,1.627858,0.983023,0.992934,1.002844,1.012755,-0.99945,-0.499725,0.0,0.499725,-0.870345
3,1.715605,1.719067,1.72253,1.725992,1.729454,1.710009,1.713471,1.380987,1.380987,1.380987,...,1.627858,0.983023,0.992934,1.002844,1.012755,-0.99945,-0.499725,0.0,0.499725,-0.522207
4,1.715605,1.719067,1.72253,1.725992,1.729454,1.710047,1.713509,1.380987,1.380987,1.380987,...,1.627858,0.983023,0.992934,1.002844,1.012755,-0.99945,-0.499725,0.0,0.499725,-0.174069


In [28]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1993.363236
1,0,2022-09-28,1994.363237
2,0,2022-09-29,1993.611981
3,0,2022-09-30,1994.61198
4,1,2022-09-27,2993.365944
5,1,2022-09-28,2994.365946
6,1,2022-09-29,2993.613661
7,1,2022-09-30,2994.613661
8,2,2022-09-27,3993.364635
9,2,2022-09-28,3994.364637


We can see that now we have trained the model to predict two steps ahead instead of one. During inference, the model also predicted two points at a time until the desired forecasting horizon was reached.

**Direct**:
- Individual models are trained for each point in the forecast horizon.

<img src="imgs/direct.png" alt="direct" width="300"/>

In [29]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = DirectStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
)

In [30]:
strategy.fit(dataset)

------------------------------------Training model for horizon 1------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9930, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.143795,-1.143795,-1.143795,...,-0.877573,-1.639090,-1.629188,-1.619287,-1.609385,1.498863,-1.499863,-1.000075,-0.500287,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.143795,-1.143795,-1.143795,...,-0.763573,-1.629188,-1.619287,-1.609385,-1.599484,-1.499863,-1.000075,-0.500287,-0.000500,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.143795,-1.143795,-1.143795,...,-0.649573,-1.619287,-1.609385,-1.599484,-1.589582,-1.000075,-0.500287,-0.000500,0.499288,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.143795,-1.143795,-1.143795,...,-0.535573,-1.609385,-1.599484,-1.589582,-1.579681,-0.500287,-0.000500,0.499288,0.999076,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.143795,-1.143795,-1.143795,...,-0.421573,-1.599484,-1.589582,-1.579681,-1.569779,-0.000500,0.499288,0.999076,1.498863,-1.566621


y for training:  (9930, 1)
[[-1.70521765]
 [-1.70175528]
 [-1.69829291]
 [-1.69483054]
 [-1.69136817]] 

0:	learn: 0.9606043	test: 0.9668573	best: 0.9668573 (0)	total: 1.42ms	remaining: 1.42s
500:	learn: 0.0037506	test: 0.0038327	best: 0.0038327 (500)	total: 596ms	remaining: 593ms
999:	learn: 0.0021159	test: 0.0022825	best: 0.0022825 (999)	total: 1.18s	remaining: 0us

bestTest = 0.002282486309
bestIteration = 999

Fold 0. Score: 0.002282486308604412
0:	learn: 0.9660711	test: 0.9614294	best: 0.9614294 (0)	total: 1.25ms	remaining: 1.25s
500:	learn: 0.0038068	test: 0.0039266	best: 0.0039266 (500)	total: 602ms	remaining: 599ms
999:	learn: 0.0020537	test: 0.0022117	best: 0.0022117 (999)	total: 1.19s	remaining: 0us

bestTest = 0.002211714388
bestIteration = 999

Fold 1. Score: 0.002211714388200288
Mean score: 0.0022
Std: 0.0
------------------------------------Training model for horizon 2------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9920, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.145979,-1.145979,-1.145979,...,-0.766722,-1.633324,-1.623414,-1.613504,-1.603593,-1.499176,-0.999450,-0.499725,0.000000,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.145979,-1.145979,-1.145979,...,-0.652694,-1.623414,-1.613504,-1.603593,-1.593683,-0.999450,-0.499725,0.000000,0.499725,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.145979,-1.145979,-1.145979,...,-0.538666,-1.613504,-1.603593,-1.593683,-1.583772,-0.499725,0.000000,0.499725,0.999450,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.145979,-1.145979,-1.145979,...,-0.424639,-1.603593,-1.593683,-1.583772,-1.573862,0.000000,0.499725,0.999450,1.499176,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.145979,-1.145979,-1.145979,...,-0.310611,-1.593683,-1.583772,-1.573862,-1.563952,0.499725,0.999450,1.499176,-1.499176,-1.566621


y for training:  (9920, 1)
[[-1.70175528]
 [-1.69829291]
 [-1.69483054]
 [-1.69136817]
 [-1.6879058 ]] 

0:	learn: 0.9623502	test: 0.9630196	best: 0.9630196 (0)	total: 1.22ms	remaining: 1.22s
500:	learn: 0.0036847	test: 0.0038944	best: 0.0038944 (500)	total: 601ms	remaining: 599ms
999:	learn: 0.0020680	test: 0.0022827	best: 0.0022827 (999)	total: 1.2s	remaining: 0us

bestTest = 0.002282716871
bestIteration = 999

Fold 0. Score: 0.002282716871130022
0:	learn: 0.9624442	test: 0.9632037	best: 0.9632037 (0)	total: 1.25ms	remaining: 1.25s
500:	learn: 0.0035087	test: 0.0036912	best: 0.0036912 (500)	total: 600ms	remaining: 598ms
999:	learn: 0.0019650	test: 0.0021591	best: 0.0021591 (999)	total: 1.19s	remaining: 0us

bestTest = 0.00215910794
bestIteration = 999

Fold 1. Score: 0.0021591079397916724
Mean score: 0.0022
Std: 0.0001
------------------------------------Training model for horizon 3------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9910, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.148169,-1.148169,-1.148169,...,-0.655600,-1.627519,-1.617600,-1.607680,-1.597761,-0.999076,-0.499288,0.000500,0.500287,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.148169,-1.148169,-1.148169,...,-0.541583,-1.617600,-1.607680,-1.597761,-1.587842,-0.499288,0.000500,0.500287,1.000075,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.148169,-1.148169,-1.148169,...,-0.427565,-1.607680,-1.597761,-1.587842,-1.577923,0.000500,0.500287,1.000075,1.499863,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.148169,-1.148169,-1.148169,...,-0.313548,-1.597761,-1.587842,-1.577923,-1.568004,0.500287,1.000075,1.499863,-1.498863,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.148169,-1.148169,-1.148169,...,-0.199531,-1.587842,-1.577923,-1.568004,-1.558085,1.000075,1.499863,-1.498863,-0.999076,-1.566621


y for training:  (9910, 1)
[[-1.69829291]
 [-1.69483054]
 [-1.69136817]
 [-1.6879058 ]
 [-1.68444342]] 

0:	learn: 0.9600707	test: 0.9634615	best: 0.9634615 (0)	total: 1.35ms	remaining: 1.35s
500:	learn: 0.0035364	test: 0.0036527	best: 0.0036527 (500)	total: 596ms	remaining: 593ms
999:	learn: 0.0019679	test: 0.0021071	best: 0.0021071 (999)	total: 1.19s	remaining: 0us

bestTest = 0.002107126777
bestIteration = 999

Fold 0. Score: 0.0021071267765355958
0:	learn: 0.9626636	test: 0.9611762	best: 0.9611762 (0)	total: 1.2ms	remaining: 1.2s
500:	learn: 0.0035232	test: 0.0036120	best: 0.0036120 (500)	total: 603ms	remaining: 600ms
999:	learn: 0.0019648	test: 0.0021312	best: 0.0021312 (999)	total: 1.2s	remaining: 0us

bestTest = 0.002131204446
bestIteration = 999

Fold 1. Score: 0.0021312044459677773
Mean score: 0.0021
Std: 0.0
------------------------------------Training model for horizon 4------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9900, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.150365,-1.150365,-1.150365,...,-0.544320,-1.621673,-1.611745,-1.601817,-1.591889,-0.498975,0.001000,0.500975,1.000950,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.150365,-1.150365,-1.150365,...,-0.430350,-1.611745,-1.601817,-1.591889,-1.581962,0.001000,0.500975,1.000950,1.500926,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.150365,-1.150365,-1.150365,...,-0.316380,-1.601817,-1.591889,-1.581962,-1.572034,0.500975,1.000950,1.500926,-1.498926,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.150365,-1.150365,-1.150365,...,-0.202410,-1.591889,-1.581962,-1.572034,-1.562106,1.000950,1.500926,-1.498926,-0.998951,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.150365,-1.150365,-1.150365,...,-0.088441,-1.581962,-1.572034,-1.562106,-1.552178,1.500926,-1.498926,-0.998951,-0.498975,-1.566621


y for training:  (9900, 1)
[[-1.69483054]
 [-1.69136817]
 [-1.6879058 ]
 [-1.68444342]
 [-1.68098105]] 

0:	learn: 0.9589837	test: 0.9626489	best: 0.9626489 (0)	total: 1.14ms	remaining: 1.14s
500:	learn: 0.0035799	test: 0.0036267	best: 0.0036267 (500)	total: 597ms	remaining: 595ms
999:	learn: 0.0019922	test: 0.0021238	best: 0.0021238 (999)	total: 1.19s	remaining: 0us

bestTest = 0.002123800997
bestIteration = 999

Fold 0. Score: 0.0021238009972551165
0:	learn: 0.9619577	test: 0.9598735	best: 0.9598735 (0)	total: 1.24ms	remaining: 1.24s
500:	learn: 0.0036115	test: 0.0037193	best: 0.0037193 (500)	total: 597ms	remaining: 594ms
999:	learn: 0.0020308	test: 0.0021786	best: 0.0021786 (999)	total: 1.19s	remaining: 0us

bestTest = 0.002178623094
bestIteration = 999

Fold 1. Score: 0.0021786230943171268
Mean score: 0.0022
Std: 0.0


(10.28906512260437, <__main__.DirectStrategy at 0x311004df0>)

__We can see that `X` for training decreases with each subsequent model.__

In [31]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on step: 0 (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-0.174069


raw_ts_X on step: 1 (110,)


...

,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-0.174069


raw_ts_X on step: 3 (110,)


,value,date,id
1,1994.000000,2022-09-21,0
2,1995.000000,2022-09-22,0
3,1996.000000,2022-09-23,0
4,1997.000000,2022-09-24,0
5,1998.000000,2022-09-25,0
6,1999.000000,2022-09-26,0
7,1996.549771,2022-09-27,0
8,1994.363237,2022-09-28,0
9,1994.841520,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.174069


In [32]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1996.549771
1,0,2022-09-28,1994.363237
2,0,2022-09-29,1994.84152
3,0,2022-09-30,1991.156717
4,1,2022-09-27,2996.551933
5,1,2022-09-28,2994.365946
6,1,2022-09-29,2994.840157
7,1,2022-09-30,2991.131408
8,2,2022-09-27,3996.579405
9,2,2022-09-28,3994.364637


**- Note 1: There is an option to use "equal_train_size" option, where all models can be trained on the same X_train set, formed for the last model predicting h point. Only the target variable (y) is updated for each model, reducing the time spent generating new training sets.**

In [13]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = DirectStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
    equal_train_size=True,
)

In [14]:
strategy.fit(dataset)

------------------------------------Training model for horizon 1------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9900, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.150365,-1.150365,-1.150365,...,-0.544320,-1.621673,-1.611745,-1.601817,-1.591889,-0.498975,0.001000,0.500975,1.000950,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.150365,-1.150365,-1.150365,...,-0.430350,-1.611745,-1.601817,-1.591889,-1.581962,0.001000,0.500975,1.000950,1.500926,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.150365,-1.150365,-1.150365,...,-0.316380,-1.601817,-1.591889,-1.581962,-1.572034,0.500975,1.000950,1.500926,-1.498926,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.150365,-1.150365,-1.150365,...,-0.202410,-1.591889,-1.581962,-1.572034,-1.562106,1.000950,1.500926,-1.498926,-0.998951,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.150365,-1.150365,-1.150365,...,-0.088441,-1.581962,-1.572034,-1.562106,-1.552178,1.500926,-1.498926,-0.998951,-0.498975,-1.566621


y for training:  (9900, 1)
[[-1.70521765]
 [-1.70175528]
 [-1.69829291]
 [-1.69483054]
 [-1.69136817]] 

0:	learn: 0.9589837	test: 0.9626489	best: 0.9626489 (0)	total: 60.1ms	remaining: 1m
500:	learn: 0.0035799	test: 0.0036267	best: 0.0036267 (500)	total: 655ms	remaining: 652ms
999:	learn: 0.0019886	test: 0.0021209	best: 0.0021209 (999)	total: 1.27s	remaining: 0us

bestTest = 0.002120945034
bestIteration = 999

Fold 0. Score: 0.002120945033678485
0:	learn: 0.9619577	test: 0.9598735	best: 0.9598735 (0)	total: 2ms	remaining: 2s
500:	learn: 0.0036115	test: 0.0037193	best: 0.0037193 (500)	total: 605ms	remaining: 603ms
999:	learn: 0.0020308	test: 0.0021786	best: 0.0021786 (999)	total: 1.21s	remaining: 0us

bestTest = 0.002178625077
bestIteration = 999

Fold 1. Score: 0.002178625076936698
Mean score: 0.0021
Std: 0.0
------------------------------------Training model for horizon 2------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9900, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.150365,-1.150365,-1.150365,...,-0.544320,-1.621673,-1.611745,-1.601817,-1.591889,-0.498975,0.001000,0.500975,1.000950,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.150365,-1.150365,-1.150365,...,-0.430350,-1.611745,-1.601817,-1.591889,-1.581962,0.001000,0.500975,1.000950,1.500926,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.150365,-1.150365,-1.150365,...,-0.316380,-1.601817,-1.591889,-1.581962,-1.572034,0.500975,1.000950,1.500926,-1.498926,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.150365,-1.150365,-1.150365,...,-0.202410,-1.591889,-1.581962,-1.572034,-1.562106,1.000950,1.500926,-1.498926,-0.998951,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.150365,-1.150365,-1.150365,...,-0.088441,-1.581962,-1.572034,-1.562106,-1.552178,1.500926,-1.498926,-0.998951,-0.498975,-1.566621


y for training:  (9900, 1)
[[-1.70175528]
 [-1.69829291]
 [-1.69483054]
 [-1.69136817]
 [-1.6879058 ]] 

0:	learn: 0.9589837	test: 0.9626489	best: 0.9626489 (0)	total: 2.07ms	remaining: 2.07s
500:	learn: 0.0035799	test: 0.0036267	best: 0.0036267 (500)	total: 611ms	remaining: 609ms
999:	learn: 0.0019922	test: 0.0021238	best: 0.0021238 (999)	total: 1.21s	remaining: 0us

bestTest = 0.002123801462
bestIteration = 999

Fold 0. Score: 0.0021238014616155863
0:	learn: 0.9619577	test: 0.9598735	best: 0.9598735 (0)	total: 1.47ms	remaining: 1.47s
500:	learn: 0.0036115	test: 0.0037193	best: 0.0037193 (500)	total: 590ms	remaining: 588ms
999:	learn: 0.0020308	test: 0.0021786	best: 0.0021786 (999)	total: 1.18s	remaining: 0us

bestTest = 0.002178623015
bestIteration = 999

Fold 1. Score: 0.002178623015287188
Mean score: 0.0022
Std: 0.0
------------------------------------Training model for horizon 3------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9900, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.150365,-1.150365,-1.150365,...,-0.544320,-1.621673,-1.611745,-1.601817,-1.591889,-0.498975,0.001000,0.500975,1.000950,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.150365,-1.150365,-1.150365,...,-0.430350,-1.611745,-1.601817,-1.591889,-1.581962,0.001000,0.500975,1.000950,1.500926,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.150365,-1.150365,-1.150365,...,-0.316380,-1.601817,-1.591889,-1.581962,-1.572034,0.500975,1.000950,1.500926,-1.498926,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.150365,-1.150365,-1.150365,...,-0.202410,-1.591889,-1.581962,-1.572034,-1.562106,1.000950,1.500926,-1.498926,-0.998951,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.150365,-1.150365,-1.150365,...,-0.088441,-1.581962,-1.572034,-1.562106,-1.552178,1.500926,-1.498926,-0.998951,-0.498975,-1.566621


y for training:  (9900, 1)
[[-1.69829291]
 [-1.69483054]
 [-1.69136817]
 [-1.6879058 ]
 [-1.68444342]] 

0:	learn: 0.9589837	test: 0.9626489	best: 0.9626489 (0)	total: 1.62ms	remaining: 1.61s
500:	learn: 0.0035799	test: 0.0036267	best: 0.0036267 (500)	total: 607ms	remaining: 604ms
999:	learn: 0.0019886	test: 0.0021209	best: 0.0021209 (999)	total: 1.2s	remaining: 0us

bestTest = 0.002120944768
bestIteration = 999

Fold 0. Score: 0.0021209447682686666
0:	learn: 0.9619577	test: 0.9598735	best: 0.9598735 (0)	total: 1.38ms	remaining: 1.38s
500:	learn: 0.0036115	test: 0.0037193	best: 0.0037193 (500)	total: 603ms	remaining: 601ms
999:	learn: 0.0020308	test: 0.0021786	best: 0.0021786 (999)	total: 1.2s	remaining: 0us

bestTest = 0.002178624707
bestIteration = 999

Fold 1. Score: 0.002178624707219995
Mean score: 0.0021
Std: 0.0
------------------------------------Training model for horizon 4------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9900, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.150365,-1.150365,-1.150365,...,-0.544320,-1.621673,-1.611745,-1.601817,-1.591889,-0.498975,0.001000,0.500975,1.000950,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.150365,-1.150365,-1.150365,...,-0.430350,-1.611745,-1.601817,-1.591889,-1.581962,0.001000,0.500975,1.000950,1.500926,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.150365,-1.150365,-1.150365,...,-0.316380,-1.601817,-1.591889,-1.581962,-1.572034,0.500975,1.000950,1.500926,-1.498926,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.150365,-1.150365,-1.150365,...,-0.202410,-1.591889,-1.581962,-1.572034,-1.562106,1.000950,1.500926,-1.498926,-0.998951,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.150365,-1.150365,-1.150365,...,-0.088441,-1.581962,-1.572034,-1.562106,-1.552178,1.500926,-1.498926,-0.998951,-0.498975,-1.566621


y for training:  (9900, 1)
[[-1.69483054]
 [-1.69136817]
 [-1.6879058 ]
 [-1.68444342]
 [-1.68098105]] 

0:	learn: 0.9589837	test: 0.9626489	best: 0.9626489 (0)	total: 1.52ms	remaining: 1.52s
500:	learn: 0.0035799	test: 0.0036267	best: 0.0036267 (500)	total: 655ms	remaining: 653ms
999:	learn: 0.0019922	test: 0.0021238	best: 0.0021238 (999)	total: 1.31s	remaining: 0us

bestTest = 0.002123800997
bestIteration = 999

Fold 0. Score: 0.0021238009972551165
0:	learn: 0.9619577	test: 0.9598735	best: 0.9598735 (0)	total: 1.28ms	remaining: 1.28s
500:	learn: 0.0036115	test: 0.0037193	best: 0.0037193 (500)	total: 613ms	remaining: 610ms
999:	learn: 0.0020308	test: 0.0021786	best: 0.0021786 (999)	total: 1.23s	remaining: 0us

bestTest = 0.002178623094
bestIteration = 999

Fold 1. Score: 0.0021786230943171268
Mean score: 0.0022
Std: 0.0


(10.621765851974487, <__main__.DirectStrategy at 0x103d95930>)

In [15]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on step: 0 (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.279197,0.949622,0.959549,0.969477,0.979405,1.00095,1.500926,-1.498926,-0.998951,-0.174069


raw_ts_X on step: 1 (110,)


...

,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.507137,0.969477,0.979405,0.989333,0.999261,-1.498926,-0.998951,-0.498975,0.001,-0.174069


raw_ts_X on step: 3 (110,)


,value,date,id
1,1994.000000,2022-09-21,0
2,1995.000000,2022-09-22,0
3,1996.000000,2022-09-23,0
4,1997.000000,2022-09-24,0
5,1998.000000,2022-09-25,0
6,1999.000000,2022-09-26,0
7,1993.583571,2022-09-27,0
8,1991.035870,2022-09-28,0
9,1991.952463,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.174069


In [16]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1993.583571
1,0,2022-09-28,1991.03587
2,0,2022-09-29,1991.952463
3,0,2022-09-30,1991.156717
4,1,2022-09-27,2993.558036
5,1,2022-09-28,2991.00616
6,1,2022-09-29,2991.926037
7,1,2022-09-30,2991.131408
8,2,2022-09-27,3993.548053
9,2,2022-09-28,3990.99943


**- Note 2: Direct can also be combined with MIMO, where each individual model predicts model_horizon points ahead.**

In [17]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = DirectStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
    model_horizon=2,
)

In [18]:
strategy.fit(dataset)

------------------------------------Training model for horizon 1------------------------------------
raw_ts_X_for_training:  (10000, 16)
X for training:  (9920, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.145979,-1.145979,-1.145979,...,-0.766722,-1.633324,-1.623414,-1.613504,-1.603593,-1.499176,-0.999450,-0.499725,0.000000,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.145979,-1.145979,-1.145979,...,-0.652694,-1.623414,-1.613504,-1.603593,-1.593683,-0.999450,-0.499725,0.000000,0.499725,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.145979,-1.145979,-1.145979,...,-0.538666,-1.613504,-1.603593,-1.593683,-1.583772,-0.499725,0.000000,0.499725,0.999450,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.145979,-1.145979,-1.145979,...,-0.424639,-1.603593,-1.593683,-1.583772,-1.573862,0.000000,0.499725,0.999450,1.499176,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.145979,-1.145979,-1.145979,...,-0.310611,-1.593683,-1.583772,-1.573862,-1.563952,0.499725,0.999450,1.499176,-1.499176,-1.566621


y for training:  (9920, 2)
[[-1.70521765 -1.70175528]
 [-1.70175528 -1.69829291]
 [-1.69829291 -1.69483054]
 [-1.69483054 -1.69136817]
 [-1.69136817 -1.6879058 ]] 

0:	learn: 1.3609687	test: 1.3619154	best: 1.3619154 (0)	total: 2.31ms	remaining: 2.31s
500:	learn: 0.0052110	test: 0.0055076	best: 0.0055076 (500)	total: 809ms	remaining: 806ms
999:	learn: 0.0029246	test: 0.0032282	best: 0.0032282 (999)	total: 1.58s	remaining: 0us

bestTest = 0.003228249025
bestIteration = 999

Fold 0. Score: 0.0032282490248902312
0:	learn: 1.3611017	test: 1.3621758	best: 1.3621758 (0)	total: 1.91ms	remaining: 1.91s
500:	learn: 0.0049621	test: 0.0052201	best: 0.0052201 (500)	total: 809ms	remaining: 806ms
999:	learn: 0.0027790	test: 0.0030534	best: 0.0030534 (999)	total: 1.61s	remaining: 0us

bestTest = 0.00305343819
bestIteration = 999

Fold 1. Score: 0.0030534381902755756
Mean score: 0.0031
Std: 0.0001
------------------------------------Training model for horizon 2------------------------------------
raw_

,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.150365,-1.150365,-1.150365,...,-0.544320,-1.621673,-1.611745,-1.601817,-1.591889,-0.498975,0.001000,0.500975,1.000950,-1.566621
1,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.150365,-1.150365,-1.150365,...,-0.430350,-1.611745,-1.601817,-1.591889,-1.581962,0.001000,0.500975,1.000950,1.500926,-1.566621
2,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.150365,-1.150365,-1.150365,...,-0.316380,-1.601817,-1.591889,-1.581962,-1.572034,0.500975,1.000950,1.500926,-1.498926,-1.566621
3,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.150365,-1.150365,-1.150365,...,-0.202410,-1.591889,-1.581962,-1.572034,-1.562106,1.000950,1.500926,-1.498926,-0.998951,-1.566621
4,-1.715605,-1.712142,-1.708680,-1.705218,-1.701755,-1.698293,-1.694831,-1.150365,-1.150365,-1.150365,...,-0.088441,-1.581962,-1.572034,-1.562106,-1.552178,1.500926,-1.498926,-0.998951,-0.498975,-1.566621


y for training:  (9900, 2)
[[-1.69829291 -1.69483054]
 [-1.69483054 -1.69136817]
 [-1.69136817 -1.6879058 ]
 [-1.6879058  -1.68444342]
 [-1.68444342 -1.68098105]] 

0:	learn: 1.3562078	test: 1.3613912	best: 1.3613912 (0)	total: 2.71ms	remaining: 2.71s
500:	learn: 0.0050627	test: 0.0051289	best: 0.0051289 (500)	total: 793ms	remaining: 790ms
999:	learn: 0.0028123	test: 0.0029995	best: 0.0029995 (999)	total: 1.64s	remaining: 0us

bestTest = 0.002999468753
bestIteration = 999

Fold 0. Score: 0.002999468752666121
0:	learn: 1.3604136	test: 1.3574661	best: 1.3574661 (0)	total: 2.21ms	remaining: 2.21s
500:	learn: 0.0051074	test: 0.0052598	best: 0.0052598 (500)	total: 786ms	remaining: 783ms
999:	learn: 0.0028720	test: 0.0030810	best: 0.0030810 (999)	total: 1.59s	remaining: 0us

bestTest = 0.003081039468
bestIteration = 999

Fold 1. Score: 0.0030810394677772894
Mean score: 0.003
Std: 0.0


(6.974285125732422, <__main__.DirectStrategy at 0x103d958a0>)

In [19]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on step: 0 (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.393167,0.959549,0.969477,0.979405,0.989333,1.500926,-1.498926,-0.998951,-0.498975,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.393167,0.959549,0.969477,0.979405,0.989333,1.500926,-1.498926,-0.998951,-0.498975,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.393167,0.959549,0.969477,0.979405,0.989333,1.500926,-1.498926,-0.998951,-0.498975,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.393167,0.959549,0.969477,0.979405,0.989333,1.500926,-1.498926,-0.998951,-0.498975,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.393167,0.959549,0.969477,0.979405,0.989333,1.500926,-1.498926,-0.998951,-0.498975,-0.174069


raw_ts_X on step: 1 (110,)


,value,date,id
1,1994.000000,2022-09-21,0
2,1995.000000,2022-09-22,0
3,1996.000000,2022-09-23,0
4,1997.000000,2022-09-24,0
5,1998.000000,2022-09-25,0
6,1999.000000,2022-09-26,0
7,1993.363236,2022-09-27,0
8,1994.363237,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (10, 28)


,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0,date__season_y__standard_scaler__lag_3,date__season_y__standard_scaler__lag_2,date__season_y__standard_scaler__lag_1,...,date__season_d__standard_scaler__lag_0,date__season_doy__standard_scaler__lag_3,date__season_doy__standard_scaler__lag_2,date__season_doy__standard_scaler__lag_1,date__season_doy__standard_scaler__lag_0,date__season_wd__standard_scaler__lag_3,date__season_wd__standard_scaler__lag_2,date__season_wd__standard_scaler__lag_1,date__season_wd__standard_scaler__lag_0,id__label__standard_scaler__lag_0
0,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-1.566621
1,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-1.218483
2,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.870345
3,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.522207
4,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454,1.375134,1.375134,1.375134,...,1.621107,0.979405,0.989333,0.999261,1.009188,-0.998951,-0.498975,0.001,0.500975,-0.174069


In [20]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1993.363236
1,0,2022-09-28,1994.363237
2,0,2022-09-29,1990.156503
3,0,2022-09-30,1991.156504
4,1,2022-09-27,2993.365944
5,1,2022-09-28,2994.365946
6,1,2022-09-29,2990.130077
7,1,2022-09-30,2991.130078
8,2,2022-09-27,3993.364635
9,2,2022-09-28,3994.364637


**FlatWideMIMO**:
- A hybrid of Direct and MIMO. One model is trained, but Direct’s features are deployed across the forecast horizon.
- Note 1: To use FlatWideMIMO with date-related features, h lags of them must be included (with help of LagTransformer).

<img src="imgs/flatwidemimo.png" alt="flatwidemimo" width="300"/>

In [21]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = FlatWideMIMOStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
)

In [22]:
strategy.fit(dataset)

raw_ts_X_for_training:  (10000, 16)


,value,date,id,value__standard_scaler,date__season_y,date__season_m,date__season_d,date__season_doy,date__season_wd,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,id__label,id__label__standard_scaler
0,1000.0,2020-01-01,0,-1.729454,2020,1,5,5,6,-1.150365,-1.557466,-1.228139,-1.651456,1.500926,0,-1.566621
1,1001.0,2020-01-02,0,-1.725992,2020,1,6,6,0,-1.150365,-1.557466,-1.114169,-1.641528,-1.498926,0,-1.566621
2,1002.0,2020-01-03,0,-1.722530,2020,1,7,7,1,-1.150365,-1.557466,-1.000199,-1.631600,-0.998951,0,-1.566621
3,1003.0,2020-01-04,0,-1.719067,2020,1,8,8,2,-1.150365,-1.557466,-0.886229,-1.621673,-0.498975,0,-1.566621
4,1004.0,2020-01-05,0,-1.715605,2020,1,9,9,3,-1.150365,-1.557466,-0.772259,-1.611745,0.001000,0,-1.566621


X for training:  (39600, 14)


,ID,FH,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0
0,0.0,1.0,-1.150365,-1.557466,-0.886229,-1.621673,-0.498975,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680
1,0.0,2.0,-1.150365,-1.557466,-0.772259,-1.611745,0.001000,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680
2,0.0,3.0,-1.150365,-1.557466,-0.658290,-1.601817,0.500975,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680
3,0.0,4.0,-1.150365,-1.557466,-0.544320,-1.591889,1.000950,-1.729454,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680
4,0.0,1.0,-1.150365,-1.557466,-0.772259,-1.611745,0.001000,-1.725992,-1.722530,-1.719067,-1.715605,-1.712142,-1.708680,-1.705218


y for training:  (39600, 1)
[[-1.70521765]
 [-1.70175528]
 [-1.69829291]
 [-1.69483054]
 [-1.70175528]] 

0:	learn: 0.9586446	test: 0.9618324	best: 0.9618324 (0)	total: 2.07ms	remaining: 2.07s
500:	learn: 0.0041781	test: 0.0042034	best: 0.0042034 (500)	total: 834ms	remaining: 831ms
999:	learn: 0.0029084	test: 0.0029512	best: 0.0029512 (999)	total: 1.68s	remaining: 0us

bestTest = 0.002951171897
bestIteration = 999

Fold 0. Score: 0.002951171896897575
0:	learn: 0.9617331	test: 0.9587321	best: 0.9587321 (0)	total: 2.69ms	remaining: 2.69s
500:	learn: 0.0043870	test: 0.0044545	best: 0.0044545 (500)	total: 873ms	remaining: 869ms
999:	learn: 0.0031491	test: 0.0032166	best: 0.0032166 (999)	total: 1.75s	remaining: 0us

bestTest = 0.003216634081
bestIteration = 999

Fold 1. Score: 0.0032166340805951004
Mean score: 0.0031
Std: 0.0001


(3.7139110565185547, <__main__.FlatWideMIMOStrategy at 0x16e457b80>)

In [23]:
forecast_time, current_pred = strategy.predict(dataset)

raw_ts_X on step: 0 (110,)


,value,date,id
1,1994.0,2022-09-21,0
2,1995.0,2022-09-22,0
3,1996.0,2022-09-23,0
4,1997.0,2022-09-24,0
5,1998.0,2022-09-25,0
6,1999.0,2022-09-26,0
7,NaN,2022-09-27,0
8,NaN,2022-09-28,0
9,NaN,2022-09-29,0
10,NaN,2022-09-30,0


X for testing:  (40, 14)


,ID,FH,date__season_y__standard_scaler,date__season_m__standard_scaler,date__season_d__standard_scaler,date__season_doy__standard_scaler,date__season_wd__standard_scaler,value__standard_scaler__lag_6,value__standard_scaler__lag_5,value__standard_scaler__lag_4,value__standard_scaler__lag_3,value__standard_scaler__lag_2,value__standard_scaler__lag_1,value__standard_scaler__lag_0
0,0.0,1.0,1.375134,0.870384,1.279197,0.979405,-0.998951,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454
1,0.0,2.0,1.375134,0.870384,1.393167,0.989333,-0.498975,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454
2,0.0,3.0,1.375134,0.870384,1.507137,0.999261,0.001000,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454
3,0.0,4.0,1.375134,0.870384,1.621107,1.009188,0.500975,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454
4,1.0,1.0,1.375134,0.870384,1.279197,0.979405,-0.998951,1.70868,1.712142,1.715605,1.719067,1.72253,1.725992,1.729454


In [24]:
current_pred.head(10)

,id,date,value
0,0,2022-09-27,1995.401964
1,0,2022-09-28,1996.039777
2,0,2022-09-29,1996.568991
3,0,2022-09-30,1992.505841
4,1,2022-09-27,2995.402058
5,1,2022-09-28,2996.03987
6,1,2022-09-29,2996.569085
7,1,2022-09-30,2992.505934
8,2,2022-09-27,3995.404626
9,2,2022-09-28,3996.042439


We can see that an additional column, `FH`, appears in the feature matrix, indicating how many steps ahead the model needs to predict.